## **Importing Modules**

In [1]:
import warnings
warnings.filterwarnings("ignore")
import keras
from keras.layers import Conv2D, Dense, MaxPooling2D, Dropout, Flatten, BatchNormalization
from keras.models import Sequential
from keras.callbacks import EarlyStopping, ReduceLROnPlateau
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam
from keras.utils import multi_gpu_model

Using TensorFlow backend.


In [2]:
print("GPUs: ")
print(keras.backend.tensorflow_backend._get_available_gpus())

GPUs: 
['/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1']


## **Setting Parameters**

In [3]:
nb_classes = 5
nb_epochs = 100
batch_size = 64
img_len = 150
img_width = 150
channels = 3

## **Data Loading and Preprocessing**

The [flowers data](https://www.kaggle.com/alxmamaev/flowers-recognition) is divided into five classes: chamomile, tulip, rose, sunflower, dandelion. The data collection is based on scraped data from flickr, google images, and yandex images.

In [4]:
datagen = ImageDataGenerator(
            rescale=1./255.0,
            shear_range=0.2,
            zoom_range=0.2,
            horizontal_flip=True,
            vertical_flip = True,
            validation_split=0.3)

In [5]:
train_generator = datagen.flow_from_directory(
    "data",
    target_size=(img_len, img_width),
    batch_size=batch_size,
    class_mode="categorical",
    subset="training")

Found 3028 images belonging to 5 classes.


In [6]:
validation_generator = datagen.flow_from_directory(
    "data",
    target_size=(img_len, img_width),
    batch_size=batch_size,
    class_mode="categorical",
    subset="validation")

Found 1295 images belonging to 5 classes.


## **Model Architecture**

In [71]:
## Convolutional Model
model = Sequential()

#1st set of Convolutions
model.add(Conv2D(filters=128, kernel_size=(3, 3), activation="relu", kernel_initializer="he_uniform", input_shape = (img_len, img_width, channels)))
model.add((BatchNormalization()))
model.add(Conv2D(filters=128, kernel_size=(3, 3), activation="relu", kernel_initializer="he_uniform"))
model.add((BatchNormalization()))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

#2nd set of Convolutions
model.add(Conv2D(filters=64, kernel_size=(3, 3), activation="relu", kernel_initializer="he_uniform"))
model.add((BatchNormalization()))
model.add(Conv2D(filters=64, kernel_size=(3, 3), activation="relu", kernel_initializer="he_uniform"))
model.add((BatchNormalization()))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

#3rd set of Convolutions
model.add(Conv2D(filters=32, kernel_size=(3, 3), activation="relu", kernel_initializer="he_uniform"))
model.add(BatchNormalization())
model.add(Conv2D(filters=32, kernel_size=(3, 3), activation="relu", kernel_initializer="he_uniform"))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

#DNN
model.add(Flatten())
model.add(Dense(units = 256, activation="relu", kernel_initializer="he_uniform"))
model.add(BatchNormalization())
model.add(Dropout(0.25))
model.add(Dense(nb_classes, activation="softmax"))
print(model.summary())



_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_64 (Conv2D)           (None, 148, 148, 128)     3584      
_________________________________________________________________
batch_normalization_75 (Batc (None, 148, 148, 128)     512       
_________________________________________________________________
conv2d_65 (Conv2D)           (None, 146, 146, 128)     147584    
_________________________________________________________________
batch_normalization_76 (Batc (None, 146, 146, 128)     512       
_________________________________________________________________
max_pooling2d_34 (MaxPooling (None, 73, 73, 128)       0         
_________________________________________________________________
dropout_45 (Dropout)         (None, 73, 73, 128)       0         
_________________________________________________________________
conv2d_66 (Conv2D)           (None, 71, 71, 64)        73792     
__________

In [73]:
# Early stopping, LR Annealer
red_lr= ReduceLROnPlateau(monitor='val_loss',patience=3,verbose=1,factor=0.1)
early_stop = EarlyStopping(monitor = "val_loss", patience = 5)

In [74]:
# Optimizer
opt = Adam(lr=0.01)

In [75]:
## Compiling the Model
parallel_model = multi_gpu_model(model, gpus=2)
parallel_model.compile(loss = "categorical_crossentropy", optimizer="adam", metrics = ["accuracy"])

## **Model Training**

In [76]:
history = parallel_model.fit_generator(train_generator, validation_data=validation_generator, epochs= nb_epochs, callbacks=[early_stop, red_lr], shuffle=True)

Epoch 1/100
48/48 [==============================] - 50s 1s/step - loss: 1.5537 - acc: 0.4431 - val_loss: 1.9785 - val_acc: 0.4764
Epoch 2/100
48/48 [==============================] - 34s 712ms/step - loss: 1.1823 - acc: 0.5392 - val_loss: 1.4860 - val_acc: 0.5459
Epoch 3/100
48/48 [==============================] - 36s 744ms/step - loss: 1.0895 - acc: 0.5781 - val_loss: 1.7070 - val_acc: 0.5236
Epoch 4/100
48/48 [==============================] - 35s 719ms/step - loss: 1.0553 - acc: 0.5909 - val_loss: 1.2928 - val_acc: 0.5776
Epoch 5/100
48/48 [==============================] - 35s 734ms/step - loss: 0.9568 - acc: 0.6306 - val_loss: 1.0602 - val_acc: 0.6131
Epoch 6/100
48/48 [==============================] - 35s 721ms/step - loss: 0.9110 - acc: 0.6436 - val_loss: 1.2266 - val_acc: 0.5884
Epoch 7/100
48/48 [==============================] - 35s 728ms/step - loss: 0.8754 - acc: 0.6670 - val_loss: 1.1286 - val_acc: 0.6355
Epoch 8/100
48/48 [==============================] - 38s 785ms/st

In [77]:
_, acc = parallel_model.evaluate_generator(validation_generator)
print("Validation Accuracy : ", round(acc*100, 3))

Validation Accuracy :  68.031
